<a href="https://colab.research.google.com/github/taekyungss/recommend_system/blob/main/sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
import tensorflow as tf

tf.__version__

In [ ]:
# !pip install tensorflow


In [ ]:
!pip install keras

In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequence

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
! pip install konlpy

In [ ]:
!pip install pygame
!pip install pytagcloud
!pip install simplejson

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 하나 이상 설정할 경우
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 모델의 최적의 하이퍼파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth



# ARIMA (시계열 예측)
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm


# 시간 측정을 위한 시간 모듈
import datetime

# 주식정보
from pandas_datareader import data

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer


# 데이터 수집
import requests
from bs4 import BeautifulSoup
import re
import time
import os
import json

# 한국어 형태소 분석
from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran

# 워드 클라우드를 위한 라이브러리
from collections import Counter
import pytagcloud
from IPython.display import Image

# 저장
import pickle

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/recommend_system/samples.csv")

In [ ]:
df

In [ ]:
df["y"] = [1 if t else 0 for t in list(df["score"]>=7)]
df

In [ ]:
df[df["y"]==0]

In [ ]:
# 한글 외에 모든 글자 제거

def text_cleaning(text):
  hangul = re.compile('[^ ㄱ-ㅣ 가-힣]+')

# 지정한 정규식에 해당하지 않는것은 길이가 0인 문자열로 반환
  result = hangul.sub("",text)
  return result

In [ ]:
df["ko_riew"] = df["sentence"].apply(lambda x: text_cleaning(x))

In [ ]:
df = df.rename(columns = {"ko_riew": "ko_review"})

In [ ]:
df

In [ ]:
# 원래 칼럼 제거

# df.drop("sentence", axis=1, inplace =True)
# df

In [ ]:
# 형태소 분석

# konlpy 라이브러리로 텍스트 데이터에서 형태소 추출

def get_pos(x):
  tagger = Okt()
  pos = tagger.pos(x)

  # 단어와 품사를 합쳐서 하나의 단어로 만들어주기
  result=[]

  # 1. 형태소의 수만큼 반복 2. 조사와 명사인 것이 같을 수 있기 때문에 구분하기 3. 형태소 벡터 만들때 추후 사용
  for a1 in pos:
    result.append(f'{a1[0]}/{a1[1]}')

  return result


In [ ]:
# 예시
get_pos(df.loc[986]["ko_review"])

In [ ]:
# 형태소 벡터 생성

index_vectorizer = CountVectorizer(tokenizer=lambda x: get_pos(x))
X = index_vectorizer.fit_transform(df["ko_review"].tolist())
X

In [ ]:
# 단어장 완성
index_vectorizer.vocabulary_

In [ ]:
print(df["ko_review"][0])
print(df["y"][0])
print(X[0])

In [ ]:
# 형태소 벡터를 TF-IDF 벡터로 생성

tfid_vectorizer = TfidfTransformer()

X = tfid_vectorizer.fit_transform(X)
print(X[0])

In [ ]:
# 학습데이터 생성

y=df["y"]

In [ ]:
#  KNN

params = {
    "n_neighbors" : list(range(1,11))
}

# 사용할 모델 객체 생성

m1 = KNeighborsClassifier()

kfold = KFold(n_splits=10, shuffle = True, random_state =1)

grid_clf1 = GridSearchCV(m1, param_grid = params, scoring="f1", cv=kfold)
grid_clf1.fit(X,y)

print(f"최적의 하이퍼 파라미터 : {grid_clf1.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf1.best_score_}")

In [ ]:
# LogisticRegression

params = {
    "penalty" : ["11", "12", "elasticnet", "none"],
    "C" : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

m2 = LogisticRegression()

grid_clf2 = GridSearchCV(m2, param_grid = params, scoring="f1", cv=kfold)
grid_clf2.fit(X,y)

print(f"최적의 하이퍼 파라미터 : {grid_clf2.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf2.best_score_}")

In [ ]:
# SVM

parmas = {
    "C" : [0.0001, 0.001, 0.01, 0.1, 1,10,100,1000,10000]
}

m3 = SVC()

grid_clf3 = GridSearchCV(m3, param_grid = params, scoring="f1", cv=kfold)
grid_clf3.fit( X, y)

print(f"최적의 하이퍼 파라미터 : {grid_clf3.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf3.best_score_}")

In [ ]:
# RandomForest
# n_estimators : 사용할 트리의 개수
# max_depth : 생성될 최대 질문 깊이, None은 무한대.

params = {
    "n_estimators": [50,100,150,200,250,300],
    "max_depth": [None, 1,2,3,4,5,6,7,8,9,10]
}

m4 = RandomForestClassifier(random_state =1)

grid_clf4 = GridSearchCV(m4, param_grid = params, scoring="f1", cv=kfold)
grid_clf4.fit( X, y)

print(f"최적의 하이퍼 파라미터 : {grid_clf4.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf4.best_score_}")


In [ ]:
# XGBoost

params  = {
    "booster": ["gbtree", "gblinear"],
    "leanring_rate": [0.0001,0.001,0.01,0.1,1,10,100,1000,10000],
    "n_estimators": [50,100,150,200,250,300]
}

m5 = XGBClassifier(silent=True, verbosity=0)

grid_clf5 = GridSearchCV(m5, param_grid = params, scoring="f1", cv=kfold)
grid_clf5.fit( X, y)

print(f"최적의 하이퍼 파라미터 : {grid_clf5.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf5.best_score_}")


In [ ]:
print(grid_clf1.best_score_)
print(grid_clf2.best_score_)
print(grid_clf3.best_score_)
print(grid_clf4.best_score_)
print(grid_clf5.best_score_) 

In [ ]:
model = grid_clf4.best_estimator_

# 상관계수 구하기
a1 = (model.coef_[0])
display(a1)
len(a1)

In [ ]:
from collections import Counter
from konlpy.tag import Okt
import pytagcloud

In [ ]:
okt = Okt()
nouns = okt.nouns(df)
print(nouns)